In [ ]:
#Instructions
#Following the class example, create a function that returns the price, names of origin and arrival airports and the name of the company. 
# Do it for all the flights between two dates that cost the same.

In [47]:
import pandas as pd
import requests

In [55]:
url = "https://partners.api.skyscanner.net/apiservices/v3/flights/indicative/search"
headers = {"x-api-key": "prtl6749387986743898559646983194"}

dates = [    {"year": 2023, "month": 5, "day": 20},    
             {"year": 2023, "month": 5, "day": 21},    
             {"year": 2023, "month": 5, "day": 22},
        ]

results = []

for date in dates:
    
    payload = {
        "query": {
            "market": "UK",
            "locale": "en-GB",
            "currency": "EUR",
            "queryLegs": [{"originPlace": {"queryPlace": {"iata": "LIS"}},                    
                           "destinationPlace": {"queryPlace": {"iata": "NYC"}},                    
                           "fixedDate": date,}],
        }
    }
    response = requests.post(url, json=payload, headers=headers)
    flight_data = response.json()
    flattened_results = []
    
    for quote_key, quote_data in flight_data['content']['results']['quotes'].items():
        
        price = quote_data['minPrice']['amount']
        
        origin_id = quote_data['outboundLeg']['originPlaceId']
        
        dest_id = quote_data['outboundLeg']['destinationPlaceId']
        
        airline_id = quote_data['outboundLeg']['marketingCarrierId']
        
        origin_name = flight_data['content']['results']['places'][origin_id]['name']
        
        dest_name = flight_data['content']['results']['places'][dest_id]['name']
        
        airline_name = flight_data['content']['results']['carriers'][airline_id]['name']
        
        flight_dict = {
                       'date': f"{date['year']}-{date['month']}-{date['day']}",
                       'price': price,
                       'origin': origin_name,
                       'destination': dest_name,
                       'company': airline_name
                       }
        
        flattened_results.append(flight_dict)
        results.append(pd.DataFrame.from_records(flattened_results))

data = pd.concat(results, ignore_index=True)
data

,date,price,origin,destination,company
0,2023-5-20,288,Lisbon,Stewart International,Iberia
1,2023-5-20,288,Lisbon,Stewart International,Iberia
2,2023-5-20,575,Lisbon,New York Newark,TAP
3,2023-5-21,544,Lisbon,New York Newark,TAP
4,2023-5-21,544,Lisbon,New York Newark,TAP
5,2023-5-21,359,Lisbon,New York John F. Kennedy,easyJet
6,2023-5-22,574,Lisbon,New York John F. Kennedy,TAP
7,2023-5-22,574,Lisbon,New York John F. Kennedy,TAP
8,2023-5-22,319,Lisbon,New York John F. Kennedy,Air Europa
